<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/secretlyincreasingstocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

기간 입력

거래대금 하위 10%~50%

등락률 상위 50%~10%

In [1]:
!pip install exchange_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import requests
import datetime
import exchange_calendars as ecals # 개장일만

In [3]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

In [4]:
today = datetime.date.today().strftime('%Y%m%d')
period = 60

In [5]:
start = (datetime.date.today() - datetime.timedelta(days=period)).strftime('%Y%m%d')

if XKRX.is_session(today) == False:
  today = XKRX.previous_open(today).strftime('%Y%m%d')
if XKRX.is_session(start) == False:
  start = XKRX.next_open(start).strftime('%Y%m%d')

In [6]:
from io import StringIO

def get_stocks(market='STK'):
  data = {
    'mktId': market,
    'strtDd': start,
    'endDd': today,
    'money': '1',
    'adjStkPrc': '2',
    'adjStkPrc_check': 'Y',
    'share': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
  }
  gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
  gen_key = requests.post(gen_url, data=data)

  down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
  r = requests.post(down_url, data={'code':gen_key.text})
  r.encoding = 'EUC-KR'

  return pd.read_csv(StringIO(r.text))

In [7]:
df = pd.concat([get_stocks(), get_stocks('KSQ')]).reset_index(drop=True)

In [8]:
df

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금
0,095570,AJ네트웍스,6800,6130,-670,-9.85,5684273,35891995370
1,006840,AK홀딩스,13250,16500,3250,24.53,797399,10425419000
2,027410,BGF,3285,4420,1135,34.55,38883540,152985578355
3,282330,BGF리테일,165500,195000,29500,17.82,2331496,429815304351
4,138930,BNK금융지주,6020,7390,1370,22.76,49282522,327141095210
...,...,...,...,...,...,...,...,...
2509,024060,흥구석유,5500,6380,880,16.00,19612003,123292230750
2510,010240,흥국,4970,6430,1460,29.38,2992010,17437908290
2511,189980,흥국에프엔비,2345,2750,405,17.27,19238528,49770441265
2512,037440,희림,7220,10900,3680,50.97,152988230,1728619462830


In [9]:
traded_df = df[(df['거래대금'] < df['거래대금'].quantile(0.40)) & (df['거래대금'] > df['거래대금'].quantile(0.2))]
traded_df

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금
1,006840,AK홀딩스,13250,16500,3250,24.53,797399,10425419000
9,00104K,CJ4우(전환),64600,68600,4000,6.19,200225,13393440000
15,097955,CJ제일제당 우,168500,169000,500,0.30,63347,10672283000
17,012030,DB,738,835,97,13.14,12464430,9655687933
18,016610,DB금융투자,4370,4620,250,5.72,3608648,15952576865
...,...,...,...,...,...,...,...,...
2491,126640,화신정공,1395,1715,320,22.94,10143298,15875094665
2495,353190,휴럼,998,1240,242,24.25,15903121,18497560600
2507,084110,휴온스글로벌,17750,19250,1500,8.45,671944,12188274150
2510,010240,흥국,4970,6430,1460,29.38,2992010,17437908290


In [10]:
increased_df = df[(df['등락률'] > df['등락률'].quantile(0.70)) & (df['등락률'] < df['등락률'].quantile(0.90))]
increased_df

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금
1,006840,AK홀딩스,13250,16500,3250,24.53,797399,10425419000
3,282330,BGF리테일,165500,195000,29500,17.82,2331496,429815304351
4,138930,BNK금융지주,6020,7390,1370,22.76,49282522,327141095210
8,079160,CJ CGV,14900,17550,2650,17.79,7867754,122259419150
16,000590,CS홀딩스,53800,62800,9000,16.73,120192,6754672600
...,...,...,...,...,...,...,...,...
2502,200670,휴메딕스,18650,22100,3450,18.50,1634364,34593190150
2503,065510,휴비츠,8430,10750,2320,27.52,3905840,40463148270
2509,024060,흥구석유,5500,6380,880,16.00,19612003,123292230750
2510,010240,흥국,4970,6430,1460,29.38,2992010,17437908290


In [11]:
import numpy as np

selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))

In [12]:
df[df['종목명'].isin(selected)].sort_values('등락률', ascending=False)

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금
2510,010240,흥국,4970,6430,1460,29.38,2992010,17437908290
712,029460,케이씨,14150,18300,4150,29.33,949227,15207150850
2390,239890,피엔에이치테크,11850,15250,3400,28.69,1274817,18220306650
1401,099390,브레인즈컴퍼니,6250,8000,1750,28.00,1762469,13222015860
859,18064K,한진칼우,22350,28300,5950,26.62,442272,13083337550
...,...,...,...,...,...,...,...,...
1136,069140,누리플랜,4445,5150,705,15.86,1778057,8448587585
570,078520,에이블씨엔씨,4550,5270,720,15.82,4360671,21023263555
1669,032080,아즈텍WB,1820,2105,285,15.66,8794285,17933090835
1889,067900,와이엔텍,7900,9130,1230,15.57,2365637,20398551200
